Vedas Jaiswal
Decision Tree from Scratch 

In [1]:
# Libraries 

from sklearn import datasets
import pandas as pd
import math as m

In [2]:
#Function to calculate log base 2
def loga(x):
    a=m.log(x)/m.log(2)
    return a

In [3]:
iris = datasets.load_iris()

In [4]:
df = pd.DataFrame(iris.data)
df.columns = ["sl", "sw", 'pl', 'pw']

In [5]:
#Function to find label for a value
#if MIN_Value <=val < (m + Mean_Value) / 2 then it is assigned label a
#if (m + Mean_Value) <=val < Mean_Value then it is assigned label b
#if (Mean_Value) <=val < (Mean_Value + MAX_Value)/2 then it is assigned label c
#if (Mean_Value + MAX_Value)/2 <=val <= MAX_Value  then it is assigned label d

def label(val, *boundaries):
    if (val < boundaries[0]):
        return 'a'
    elif (val < boundaries[1]):
        return 'b'
    elif (val < boundaries[2]):
        return 'c'
    else:
        return 'd'

#Function to convert a continuous data into labelled data
#There are 4 lables  - a, b, c, d
def toLabel(df, old_feature_name):
    second = df[old_feature_name].mean()
    minimum = df[old_feature_name].min()
    first = (minimum + second)/2
    maximum = df[old_feature_name].max()
    third = (maximum + second)/2
    return df[old_feature_name].apply(label, args= (first, second, third))

In [6]:
#Convert all columns to labelled data
df['sl_labeled'] = toLabel(df, 'sl')
df['sw_labeled'] = toLabel(df, 'sw')
df['pl_labeled'] = toLabel(df, 'pl')
df['pw_labeled'] = toLabel(df, 'pw')
df.drop(['sl', 'sw', 'pl', 'pw'], axis = 1, inplace = True)
df

,sl_labeled,sw_labeled,pl_labeled,pw_labeled
0,b,c,a,a
1,a,b,a,a
2,a,c,a,a
3,a,c,a,a
4,a,c,a,a
5,b,d,a,a
6,a,c,a,a
7,a,c,a,a
8,a,b,a,a
9,a,c,a,a


In [7]:
#New dataframe
df.head()

,sl_labeled,sw_labeled,pl_labeled,pw_labeled
0,b,c,a,a
1,a,b,a,a
2,a,c,a,a
3,a,c,a,a
4,a,c,a,a


In [8]:
#Description of the new dataframe
df.describe()

,sl_labeled,sw_labeled,pl_labeled,pw_labeled
count,150,150,150,150
unique,4,4,4,4
top,c,b,c,c
freq,53,64,63,56


In [9]:
#Function to calculate entropy:

#Since, we are using log base 2, so the entropy need necessarily not be in between 0 and 1 only, it can get greater than 1

def entropy(df):
#Total entries    
    total=df.count()[0]
#set containing the unique output
    feat=set(df['result'])
#Calculate entropy in variable in_ans
    in_ans=0
    for i in feat:
        a=df[df['result']==i].count()
        true=a[0]
        in_per=(true/total)
        if(in_per==0):
            in_ans-=0
        else:
            in_ans-=in_per*loga(in_per)
    return in_ans

In [10]:
#Print entropy and count of possible outputs when base case is encountered

def getdata(df):
    set_df=set(df['result'])
    for w in set_df:
        q=w
        count_df=df[df['result']==w].count()[0]  
        print("Count of ",q,'=',count_df)
    print("Current Entropy is:",entropy(df))

In [11]:
#Calculate Information gain and thus gain ratio
def gain(df,s_f):
#Initial Information Required    
    initial=entropy(df)
    
#After splitting    
    total=df.count()[0]
    feature=set(df[s_f])
    final=0
    split_in=0
    for i in feature:
        update_df=df[df[s_f]==i]
        count_feat=df[df[s_f]==i].count()[0]
        final+=(count_feat/total)*entropy(update_df)
        split_in-=(count_feat/total)*loga(count_feat/total)
        
    
#Information gain    
    gain_val=initial-final
    g_r=gain_val/split_in
    return g_r

In [12]:
def build_tree(df, unused_features,level):
    
    #Printing the level
    print("Level ",level)
    #base case
    # 1.contains only one distinct value
    if(len(set(df['result']))==1):
        
        #Printing the occurences of all possible outputs
        getdata(df)
        
        print("Pure Node")
        print(" ")
        return

    # 2.all features have been used
    elif(len(unused_features)==0):
        
        #Printing the occurences of all possible outputs
        getdata(df)
        
        print("No more splitting possible")
        print(" ")
        return   
    
    #Print all the counts of possible output and entropy
    
    getdata(df)
    
    #Getting the best gain ratio
    best_feature = '' #stores best feature
    max_feats=0       #stores the best gain ratio
    
    #finding maximum gain ratio,comparing all other features gain ratio
    for f in unused_features:
        gains=gain(df,f)
        if(gains>max_feats):
            max_feats=gains
            best_feature=f
    
    #Obtain the best feature and gain ratio
    print("Splitting on feature:",best_feature,"with gain ratio",max_feats)
    
    #Update unused_feature by deleting the best feature
    best_f={best_feature}
    unused_features=unused_features-best_f
    best_f={}
        
    #get all the possible unique values of that best feature column , because the splitting should be of same number as there are unique values of that best_feature
    fin_set=set(df[best_feature])
    
    print(" ")
 
    
    #Incrementing the level by 1
    level+=1
    
    #recursive call for tree
    for k in fin_set:
        new=df[df[best_feature]==k]
        #Delete best feature column
        del new[best_feature]
        build_tree(new,unused_features,level)
        
    # here you should know the best feature
    # print it out
    #print("Best Feature ", best_feature)
    # remove best feature from unused features
    #unused_features=unused_features-best_feature
    # loop over possible values of best feature
    # call build tree recursively

In [13]:
#The build_tree function does all the required work

y = pd.DataFrame(iris.target)

#unused_features has all unique possible column names
unused_features = set(df.columns)

#df is a datframe which contains both data and target together
df['result']=y

#Function is called:
build_tree(df, unused_features,0)

Level  0
Count of  0 = 50
Count of  1 = 50
Count of  2 = 50
Current Entropy is: 1.584962500721156
Splitting on feature: pw_labeled with gain ratio 0.699638203622209
 
Level  1
Count of  1 = 10
Current Entropy is: 0.0
Pure Node
 
Level  1
Count of  1 = 40
Count of  2 = 16
Current Entropy is: 0.863120568566631
Splitting on feature: pl_labeled with gain ratio 0.4334099495621066
 
Level  2
Count of  1 = 1
Current Entropy is: 0.0
Pure Node
 
Level  2
Count of  2 = 8
Current Entropy is: 0.0
Pure Node
 
Level  2
Count of  1 = 39
Count of  2 = 8
Current Entropy is: 0.6581912658132185
Splitting on feature: sl_labeled with gain ratio 0.12674503775809332
 
Level  3
Count of  1 = 14
Current Entropy is: 0.0
Pure Node
 
Level  3
Count of  2 = 1
Current Entropy is: 0.0
Pure Node
 
Level  3
Count of  1 = 2
Current Entropy is: 0.0
Pure Node
 
Level  3
Count of  1 = 23
Count of  2 = 7
Current Entropy is: 0.783776947484701
Splitting on feature: sw_labeled with gain ratio 0.07092036405148876
 
Level  4
Co